In [1]:
import folium,math,matplotlib
#from osgeo import ogr, osr
import matplotlib.pyplot as plt
import pandas as pd,json
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin
from datetime import date, datetime, timedelta
from sklearn.pipeline import Pipeline
import branca.colormap as cm
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import Point

In [124]:
###Functions from Dr. S's code

def parseTStamp(tstamp_df):    
    return datetime.strptime(tstamp_df['timeAtPosition'], '%Y-%m-%dT%H:%M:%S%z')


def dms2dec(dms_tuple):
   # print(dms_tuple)
    
    deg, minutes, seconds, direction = dms_tuple
    if np.isnan(seconds):
        seconds=0
    # print("Out", deg, minutes, seconds, direction)
    sign = 1
    direction = direction.strip()
    if direction == 'W' or direction == 'WEST' or direction =='S' or direction=='SOUTH':
        sign=-1
    return sign*(deg + minutes/60.0 + seconds/(60.0*60.0))


def simple2alt(alt_df):
    try:
        return int(alt_df['simpleAltitude'].strip('C').strip('T')) *100
    except Exception as exp:
        return alt_df['simpleAltitude']
    
    
class TFMSDataPreprocessor(BaseEstimator, TransformerMixin):
    
    def __init__(self, attributes):
        self.attributes = attributes
    
    
    def fit(self, X, y=None):
        return self
    
    
    def transform(self, X):        
        X['lat'] = X[['latitudeDMS_degrees','latitudeDMS_minutes','latitudeDMS_seconds', 'latitudeDMS_direction']].apply(dms2dec, axis=1)
        X['long'] = X[['longitudeDMS_degrees','longitudeDMS_minutes','longitudeDMS_seconds', 'longitudeDMS_direction']].apply(dms2dec, axis=1)
        X['alt'] = X[['simpleAltitude']].apply(simple2alt, axis=1)
        X['tstamp'] = X[['timeAtPosition']].apply(parseTStamp, axis=1)
        return X[self.attributes]
    

extract_attributes = [
    'tstamp',
    'flightRef',
    'acid',
    'gufi',    
    'lat',
    'long',
    'alt',
    'speed',
    'depArpt',
    'arrArpt']

tfms_csv_files = ['KDFW20190512_170000.csv','KDFW20190512_160000.csv']
tfms_df_dict={}

colors1=[]
mymap = folium.Map(location=[32.8917,-97.0417], zoom_start=9 , tiles='Stamen Terrain')

for csv in tfms_csv_files:
    #print('Read: {}'.format(csv))
    tfms_df_dict={csv:pd.read_csv(csv)}
    for key in tfms_df_dict:
        pipe = Pipeline([('SELECT', TFMSDataPreprocessor(extract_attributes)), ])
        flt_waypoints_df = pipe.fit_transform(tfms_df_dict[key])
        flt_waypoints_df=flt_waypoints_df.dropna()
        #print(flt_waypoints_df.shape)
        lats=flt_waypoints_df['lat'].tolist()
        longs=flt_waypoints_df['long'].tolist()
        altitudes=flt_waypoints_df['alt'].tolist()
        #print(csv,len(altitudes))
        tstamps=flt_waypoints_df['tstamp'].tolist()
        #print(csv,len(tstamps))
        acid=flt_waypoints_df['acid'].tolist()
        
        alt_min = np.min(altitudes)
        #print("alt_min",alt_min)
        alt_max = np.max(altitudes)
        #print("alt_max",alt_max)
        
        cmap=cm.LinearColormap(['b','c','g','y','r'], vmin=alt_min, vmax=alt_max, caption=csv)
        fg=folium.FeatureGroup(name=csv, show=False)

        
        for ii in range(0,len(altitudes)):
            #color_interp = np.interp(altitudes[ii],[alt_min,alt_max],[50,200])
            fg.add_child(folium.CircleMarker([lats[ii],longs[ii]],
                #radius=0.0005*altitudes[ii],
                radius=5,
                popup=altitudes[ii],
                          fill=True,
                fill_color=cmap(altitudes[ii]),
                weight=0, fill_opacity=1))

        #folium.Circle(location=[32.8917,-97.0417],fill_color='#000', radius=18520, weight=1, color="#000").add_to(mymap)
        #cax, _ = matplotlib.colorbar.make_axes(ax)
        #cbar = matplotlib.colorbar.ColorbarBase(cax, cmap=cmap,norm=normalize,label='Elevation')
        cmap.caption='Flight altitude'
        mymap.add_child(fg)
        mymap.add_child(cmap)
        
mymap.add_child(folium.map.LayerControl())
mymap.save('MyFoliummap.html')

In [15]:
flt_waypoints_df=flt_waypoints_df.dropna()
flt_waypoints_df.shape

(20776, 10)

In [ ]:
lats=flt_waypoints_df['lat'].tolist()
longs=flt_waypoints_df['long'].tolist()
altitudes=flt_waypoints_df['alt'].tolist()
#print(len(altitudes))
tstamps=flt_waypoints_df['tstamp'].tolist()
#print(len(tstamps))
acid=flt_waypoints_df['acid'].tolist()

In [ ]:
mymap = folium.Map(location=[32.8917,-97.0417], zoom_start=9 , tiles='Stamen Terrain')

In [ ]:
from branca.colormap import linear

# scatter map based on lons, lats, alts and color .
alt_min = np.min(altitudes)
#print("alt_min",alt_min)
alt_max = np.max(altitudes)
#print("alt_max",alt_max)

#cmap = plt.get_cmap('gist_rainbow')
cmap1=cm.LinearColormap(['b','c','g','y','r'], vmin=alt_min, vmax=alt_max, caption='KDFW20190512_160000')
cmap2=cm.LinearColormap(['b','c','g','y','r'], vmin=alt_min, vmax=alt_max, caption='KDFW20190512_170000')
#normalize = matplotlib.colors.Normalize(vmin=alt_min, vmax=alt_max)
cmap

In [ ]:
for ii in range(0,len(altitudes)):
    #color_interp = np.interp(altitudes[ii],[alt_min,alt_max],[50,200])
    folium.CircleMarker([lats[ii],longs[ii]],
        #radius=0.0005*altitudes[ii],
        radius=5,
        popup=altitudes[ii],
                  fill=True,
        fill_color=cmap(altitudes[ii]),
        weight=0, fill_opacity=1).add_to(mymap)
    
#folium.Circle(location=[32.8917,-97.0417],fill_color='#000', radius=18520, weight=1, color="#000").add_to(mymap)
#cax, _ = matplotlib.colorbar.make_axes(ax)
#cbar = matplotlib.colorbar.ColorbarBase(cax, cmap=cmap,norm=normalize,label='Elevation')
cmap.caption='Flight altitude'
mymap.add_child(cmap)
#plugins.Fullscreen(position='topright', force_separate_button=True).add_to(mymap)

In [ ]:
mymap.save('MyFoliummap.html')

In [ ]:
!pip install branca

In [ ]:
!pip install folium

In [ ]:
!pip install shapely